In [1]:
from src.requestcompletion.llm.models._litellm_wrapper import LiteLLMWrapper
from src.requestcompletion.llm import ToolCall, Tool, Parameter
from src.requestcompletion.llm.message import UserMessage, SystemMessage, AssistantMessage, ToolMessage, ToolResponse
from src.requestcompletion.llm.history import MessageHistory
from pydantic import BaseModel
import litellm
from typing import List
import src.requestcompletion as rc


In [2]:
# Initialize the LiteLLMWrapper with the desired model
litellm_model = rc.llm.AnthropicLLM(model_name="anthropic/claude-3-5-sonnet-20241022")
# litellm_model = rc.llm.OpenAILLM(model_name="openai/gpt-4o")

## General chat

In [3]:
# Create a message history
messages = MessageHistory([
    SystemMessage("You are a helpful assistant."),
    UserMessage("Can you tell me a joke?")
])

# Perform a chat completion
response = litellm_model.chat(messages)

# Print the assistant's response
print(response.message.content)

Here's a classic one:

Why don't scientists trust atoms?
Because they make up everything! 😄


## Structured output

In [4]:
class DishInfo(BaseModel):
    name: str
    cuisine: str
    calories: int

messages = MessageHistory([
    SystemMessage("You are a nutrition expert"),
    UserMessage("Tell me about a popular Italian dish."),
])

result = litellm_model.structured(messages, DishInfo)

In [5]:
result.message.content

DishInfo(name='Spaghetti Carbonara', cuisine='Italian', calories=450)

In [6]:
class Engine(BaseModel):
    manufacturing_country: str
    number_of_cylinders: int

class CarInfo(BaseModel):
    brand: str
    country: str
    engine: Engine

class Info(BaseModel):
    cars: List[CarInfo]

sports_cars_description = """
Among the world’s most thrilling sports cars, the Ferrari 812 Superfast, crafted in Italy,
features a naturally aspirated V12 engine also built in Italy, delivering blistering power
through its 12-cylinder setup. From Germany, the Porsche 911 Turbo S stands out with its
twin-turbocharged flat-six engine, manufactured in Germany, offering a perfect balance of
performance and refinement. Meanwhile, the Chevrolet Corvette Z06, proudly American-made
in the USA, houses a hand-built 5.5-liter flat-plane crank V8 engine, also produced in the
USA, boasting 8 cylinders of raw muscle. The McLaren 720S, a British marvel from the UK,
is equipped with a twin-turbo V8 engine made in England, showcasing 8 cylinders of
precision engineering. Finally, the Toyota GR Supra, originating from Japan, uses a
3.0-liter inline-6 engine built in Austria by BMW, blending Japanese design with German
power and offering 6 cylinders of smooth performance.
"""

messages = MessageHistory(
    [
        SystemMessage("You are a car enthusiast who can extract information from paragraphs about different cars."\
                                  "Ensure you analyze the whole paragraph and return information about all cars mentioned"),
        UserMessage(f"Extract the information about all of the cars mentioned in: \n {sports_cars_description}")
    ]
)

result = litellm_model.structured(messages, Info)

In [7]:
result.message.content

Info(cars=[CarInfo(brand='Ferrari 812 Superfast', country='Italy', engine=Engine(manufacturing_country='Italy', number_of_cylinders=12)), CarInfo(brand='Porsche 911 Turbo S', country='Germany', engine=Engine(manufacturing_country='Germany', number_of_cylinders=6)), CarInfo(brand='Chevrolet Corvette Z06', country='USA', engine=Engine(manufacturing_country='USA', number_of_cylinders=8)), CarInfo(brand='McLaren 720S', country='UK', engine=Engine(manufacturing_country='England', number_of_cylinders=8)), CarInfo(brand='Toyota GR Supra', country='Japan', engine=Engine(manufacturing_country='Austria', number_of_cylinders=6))])

## Stream chat

In [8]:
messages = MessageHistory([
    UserMessage("Can you tell me a joke?")
])

resp = litellm_model.stream_chat(messages)

In [9]:
resp.streamer

<generator object LiteLLMWrapper.stream_chat.<locals>.streamer at 0x0000029DDFD16B90>

In [10]:
for x in resp.streamer:
    print(x)


Here
's a classic one:

Why don't scientists
 trust atoms?
Because they make up everything
! 😄



## Tool Calling

### Tools



In [11]:
def available_locations() -> List[str]:
    """Returns a list of available locations.
    Args:
    Returns:
        List[str]: A list of available locations.
    """
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Delhi",
        "Mumbai",
        "Bangalore",
        "Paris",
        "Denmark",
        "Sweden",
        "Norway",
        "Germany",
        "Vancouver",
        "Toronto",
    ]

def currency_used(location: str) -> str:
    """Returns the currency used in a location.
    Args:
        location (str): The location to get the currency used for.
    Returns:
        str: The currency used in the location.
    """
    currency_map = {
        "New York": "USD",
        "Los Angeles": "USD",
        "Chicago": "USD",
        "Delhi": "INR",
        "Mumbai": "INR",
        "Bangalore": "INR",
        "Paris": "EUR",
        "Denmark": "EUR",
        "Sweden": "EUR",
        "Norway": "EUR",
        "Germany": "EUR",
        "Vancouver": "CAD",
        "Toronto": "CAD",
    }
    used_currency = currency_map.get(location)
    if used_currency is None:
        raise ValueError(f"Currency not available for location: {location}")
    return used_currency

def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for.
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
        "Vancouver": 200.0,
        "Toronto": 180.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days

def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    """Converts currency using a static exchange rate (for testing purposes).
    Args:
        amount (float): The amount to convert.
        from_currency (str): The currency to convert from.
        to_currency (str): The currency to convert to.
    Returns:
        float: The converted amount.
    Raises:
        ValueError: If the exchange rate is not available.
    """
    exchange_rates = {
        ("USD", "EUR"): 0.85,
        ("EUR", "USD"): 1.1765,
        ("USD", "INR"): 83.0,
        ("INR", "USD"): 0.01205,
        ("EUR", "INR"): 98.0,
        ("INR", "EUR"): 0.0102,
        ("CAD", "USD"): 0.78,
        ("USD", "CAD"): 1.28,
        ("CAD", "EUR"): 0.66,
        ("EUR", "CAD"): 1.52,
        ("INR", "CAD"): 0.0125,
        ("CAD", "INR"): 80.0,
    }

    rate = exchange_rates.get((from_currency, to_currency))
    if rate is None:
        raise ValueError("Exchange rate not available")
    return amount * rate

In [12]:
import json
import litellm

# model = "gpt-3.5-turbo-1106"
# model = "gpt-4o"
model = "claude-3-sonnet-20240229"

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "75", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for, e.g. "New York"
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
        "Vancouver": 200.0,
        "Toronto": 180.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days


# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": "Give me a summary for 2 day trip to New York. Use NewYork as an arg if ou are tool_calling"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "average_location_cost",
            "description": "Get the average cost of living in a location for a given number of days",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location to get the cost of living for.",
                    },
                    "num_days": {"type": "integer", "description": "The number of days for the trip"},
                },
                "required": ["location", "num_days"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = litellm.completion(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
print("\nFirst LLM Response:\n", response)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

print("\nLength of tool calls", len(tool_calls))



First LLM Response:
 ModelResponse(id='chatcmpl-9223ee9a-09c0-4884-b229-ff1049149104', created=1748370309, model='claude-3-sonnet-20240229', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content="Here's a summary for a 2-day trip to New York City:", role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=1, function=Function(arguments='{"location": "New York", "num_days": 2}', name='average_location_cost'), id='toolu_01CfLVYLrVG8ovv7rgaZJPY4', type='function')], function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=92, prompt_tokens=397, total_tokens=489, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

Length of tool calls 1


In [13]:
# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
        "average_location_cost": average_location_cost,
    }  
    messages.append(response_message)  # extend conversation with assistant's reply

    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        print(function_args)
        function_response = function_to_call(**function_args)
        print(function_response)

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )  # extend conversation with function response

    

{'location': 'New York', 'num_days': 2}
400.0


In [14]:
print(messages[0])


{'role': 'user', 'content': 'Give me a summary for 2 day trip to New York. Use NewYork as an arg if ou are tool_calling'}


In [15]:
print(messages)
second_response = litellm.completion(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)  # get a new response from the model where it can see the function response
print("\nSecond LLM response:\n", second_response)

[{'role': 'user', 'content': 'Give me a summary for 2 day trip to New York. Use NewYork as an arg if ou are tool_calling'}, Message(content="Here's a summary for a 2-day trip to New York City:", role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=1, function=Function(arguments='{"location": "New York", "num_days": 2}', name='average_location_cost'), id='toolu_01CfLVYLrVG8ovv7rgaZJPY4', type='function')], function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}), {'tool_call_id': 'toolu_01CfLVYLrVG8ovv7rgaZJPY4', 'role': 'tool', 'name': 'average_location_cost', 'content': '400.0'}]


c:\Users\Aryan\dev\rc\myenv\lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'index': 1, 'function': ...Y4', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(



Second LLM response:
 ModelResponse(id='chatcmpl-dc4ead0e-f274-446d-b245-41007eeefd05', created=1748370311, model='claude-3-sonnet-20240229', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content='Based on the average location cost tool, a 2-day trip to New York City for one person would cost around $400 for accommodation, meals, and basic expenses.', role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=1, function=Function(arguments='{"location": "New York, NY", "unit": "fahrenheit"}', name='get_current_weather'), id='toolu_01ASKobiYWV23cfA7fvS3XbZ', type='function')], function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usage=Usage(completion_tokens=114, prompt_tokens=503, total_tokens=617, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creatio

### RC: chat_with_tools invokation

In [16]:
tool_instances = [
    Tool(
        name="average_location_cost",
        detail="Get the average cost of living in a location for a given number of days",
        parameters={
            Parameter(
                name="location",
                param_type="string",
                description="The location to get the cost of living for.",
                required=True
            ),
            Parameter(
                name="num_days",
                param_type="integer",
                description="The number of days for the trip",
                required=True
            )
        }
    ),
    Tool(
        name="get_current_weather",
        detail="Get the current weather in a given location",
        parameters={
            Parameter(
                name="location",
                param_type="string",
                description="The city and state, e.g. San Francisco, CA",
                required=True
            ),
            Parameter(
                name="unit",
                param_type="string",
                description="The unit to use for temperature (celsius or fahrenheit)",
                required=False
            )
        }
    )
]

In [17]:
messages = MessageHistory([SystemMessage("You are a travel agent with access to the following tools: get_current_weather, avergae_location_cost"),
                           UserMessage("Give me a summary for 2 day trip to New York. Use New York as an arg if ou are tool_calling"),
                          ])
first_response = litellm_model.chat_with_tools(messages, tool_instances)

In [18]:
print("\nFirst LLM Response:\n", first_response)
response_message = first_response.message.content
tool_calls = [tool_call for tool_call in response_message if isinstance(tool_call, ToolCall)]

print("\nLength of tool calls", len(tool_calls))

if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
        "average_location_cost": average_location_cost,
    }  
    messages.append(AssistantMessage(response_message))  # extend conversation with assistant's reply

    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.name
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**tool_call.arguments)

        messages.append(
            ToolMessage(content=ToolResponse(identifier=tool_call.identifier, result=str(function_response), name=function_name))
        )
print("After Tool Calls: ", messages)

second_response = litellm_model.chat_with_tools(messages, tool_instances)
print("\nSecond LLM response:\n", second_response)


First LLM Response:
 assistant: [ToolCall(identifier='toolu_019FQBKuezeeTn18nFmkKaTa', name='get_current_weather', arguments={'location': 'New York'}), ToolCall(identifier='toolu_01PcWTkgbfphQFGwaozdLi13', name='average_location_cost', arguments={'location': 'New York', 'num_days': 2})]

Length of tool calls 2
After Tool Calls:  system: You are a travel agent with access to the following tools: get_current_weather, avergae_location_cost
user: Give me a summary for 2 day trip to New York. Use New York as an arg if ou are tool_calling
assistant: [ToolCall(identifier='toolu_019FQBKuezeeTn18nFmkKaTa', name='get_current_weather', arguments={'location': 'New York'}), ToolCall(identifier='toolu_01PcWTkgbfphQFGwaozdLi13', name='average_location_cost', arguments={'location': 'New York', 'num_days': 2})]
tool: get_current_weather -> {"location": "New York", "temperature": "75", "unit": "fahrenheit"}
tool: average_location_cost -> 400.0


c:\Users\Aryan\dev\rc\myenv\lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...Ta', 'type': 'function'}, input_type=dict])
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...13', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(



Second LLM response:
 assistant: Based on the data I've gathered, here's a summary for your 2-day trip to New York:

Weather:
- Current temperature in New York is 75°F, which is quite pleasant for exploring the city.

Cost:
- The average cost for a 2-day stay in New York would be approximately $400. This typically includes basic accommodations and daily expenses.

Keep in mind that:
1. The weather can change, so it's good to check closer to your travel date
2. The cost is an average estimate - actual expenses can vary depending on your choice of accommodations, dining preferences, and activities
3. New York can be quite expensive compared to many other cities, so it's good to budget accordingly

Would you like any specific information about attractions or areas to stay in New York?


# LiteLLM integrated RC

In [19]:
from src.requestcompletion import llm
import src.requestcompletion as rc
from pydantic import BaseModel, Field

In [20]:
class Integrate(BaseModel):
    integrand: str = Field(description="The function you want to integrate in Latex")
    lower_limit: float = Field(description="The lower limit of the integral")
    upper_limit: float = Field(description="The upper limit of the integral")


class Simplify(BaseModel):
    expression: str = Field(description="The expression you want to simplify in Latex")


class FinalResponse(BaseModel):
    result: str = Field(description="The result of the tool call")
    key_challenges: str = Field(
        description="The key challenges faced encountered during the tool call"
    )


tools = [
    llm.Tool(
        name="integrate",
        detail="A tool to evaluate math expressions",
        parameters=Integrate,
    ),
    llm.Tool(
        name="simplify", detail="A tool to evaluate math expressions", parameters=Simplify
    ),
]


In [21]:
class integrate_tool(rc.Node):
    def __init__(self, integrand, lower_limit, upper_limit):
        super().__init__()

    async def invoke(self) -> float:
        return 1.5
    
    @classmethod
    def tool_info(self) -> rc.llm.Tool:
        return tools[0]
    
    @classmethod
    def pretty_name(self) -> str:
        return "integrate"

class simplify_tool(rc.Node):
    def __init__(self, expression):
        super().__init__()

    async def invoke(self) -> str:
        return "x + 1"
    
    @classmethod
    def tool_info(self) -> rc.llm.Tool:
        return tools[1]
    
    @classmethod
    def pretty_name(self) -> str:
        return "simplify"

In [22]:
def integrate(integrand: str, lower_limit: float, upper_limit: float) -> float:
    """Integrate a function from lower limit to upper limit
    Args:
        integrand (str): The function you want to integrate in Latex
        lower_limit (float): The lower limit of the integral
        upper_limit (float): The upper limit of the integral
    Returns:
        float: The result of the integration
    """

    return 1.5

def simplify(expression: str) -> str:
    """Simplify an expression
    Args:
        expression (str): The expression you want to simplify in Latex
    Returns:
        str: The simplified expression
    """
    return "x + 1"

In [23]:
math_agent = rc.library.tool_call_llm(
                                      connected_nodes=[integrate_tool, simplify_tool],
                                      # connected_nodes=[rc.library.from_function(integrate), rc.library.from_function(simplify)],
                                      pretty_name="Math Agent",
                                      model=rc.llm.AnthropicLLM("claude-3-sonnet-20240229"),
                                      # model=rc.llm.OpenAILLM("gpt-4o"),
                                      system_message=rc.llm.SystemMessage("You are a helpful assistant that can use tools to help the user."),
                                      output_model=FinalResponse
                                    )

In [24]:
message_history = llm.MessageHistory(
    [
        llm.UserMessage("What is the integral of x^2/x + 1 from [0, 1]?"),
    ]
)

response = await rc.call(math_agent, message_history=message_history)


[+23.486 s] RC.RUNNER   : INFO     - START CREATED Math Agent - (, message_history=user: What is the integral of x^2/x + 1 from [0, 1]?)
[+25.802 s] RC.RUNNER   : INFO     - Math Agent CREATED integrate - ({'integrand': '\\frac{x^2}{x+1}', 'lower_limit': 0, 'upper_limit': 1}, )
[+25.807 s] RC.RUNNER   : INFO     - integrate DONE 1.5
c:\Users\Aryan\dev\rc\myenv\lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...GD', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(
[+27.963 s] RC.RUNNER   : INFO     - Math Agent CREATED FinalResponse - (, message_history=user: system: You are a helpful assistant that can use tools to help the user.
user: What is the integral of x^2/x + 1 from [0, 1]?
assistant: [ToolCall(identifier='toolu_013MDsF2SinCQYJCixBFWMGD', name='int

In [25]:
assert isinstance(response, FinalResponse)
print(response.result)
print(response.key_challenges)

1.5
Integrating a rational function required using a symbolic integration tool.
